# MulNode

In [25]:
class MulNode:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y
        
        return out 
    
    def backward(self, dout):
        dx = dout*self.y
        dy = dout*self.x
        
        return dx, dy

In [16]:
# 순전파

apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulNode()
mul_tax_layer = MulNode()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(int(price))

# 역전파

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, int(dapple_num), dtax)

220
2.2 110 200


In [30]:
# 순전파

Input_1 = 100
Input_2 = 2
Input_3 = 1.1

MulLayer_1 = MulNode()
MulLayer_2 = MulNode()

Output_1 = MulLayer_1.forward(Input_1, Input_2)
Output_2 = MulLayer_2.forward(Output_1, Input_3)

print(int(Output_2))

# 역전파

dOutput_2 = 1
dOutput_1, dInput_3 = MulLayer_2.backward(dOutput_2)
dInput_1, dInput_2 = MulLayer_1.backward(dOutput_1)

print(dInput_1, int(dInput_2), dInput_3)

220
2.2 110 200


# AddNode

In [26]:
class AddNode:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        return x+y
    
    def backward(self, dout):
        return dout*1, dout*1

In [20]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulNode()
mul_orange_layer = MulNode()
add_apple_orange_layer = AddNode()
mul_tax_layer = MulNode()

apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(int(price))
print(int(dapple_num), dapple, int(dorange), int(dorange_num), dtax)

715
110 2.2 3 165 650


In [29]:
Input_1 = 100
Input_2 = 2
Input_3 = 150
Input_4 = 3
Input_5 = 1.1

MulNode_1 = MulNode()
MulNode_2 = MulNode()
AddNode_1 = AddNode()
MulNode_3 = MulNode()

Output_1 = MulNode_1.forward(Input_1, Input_2)
Output_2 = MulNode_2.forward(Input_3, Input_4)
Output_3 = AddNode_1.forward(Output_1, Output_2)
Output_4 = MulNode_3.forward(Output_3, Input_5)

dOutput_4 = 1
dOutput_3, dInput_5 = MulNode_3.backward(dOutput_4)
dOutput_1, dOutput_2 = AddNode_1.backward(dOutput_3)
dInput_3, dInput_4 = MulNode_2.backward(dOutput_2)
dInput_1, dInput_2 = MulNode_1.backward(dOutput_1)

print(int(price))
print(int(dInput_2), dInput_1, int(dInput_3), int(dInput_4), dInput_5)

715
110 2.2 3 165 650


# ReLU

In [31]:
class Relu:
    def __init__(self):
        self.mask = None 
        
    def forward(self, x): # x: numpy.ndarray 타입
        self.mask = (x<=0) # True or False. numpy.ndarray 타입. elementwise 연산.
        out = x.copy()
        out[self.mask] = 0 # 0 이하의 값은 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0 # 순전파 때의 값이 0 이하면 역전파 때는 하류로 전달되는 값이 없도록(0이 되도록)함
        dx = dout
        
        return dx

# Sigmoid

In [32]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out # (dL/dy)*(1-y)*y
        
        return dx

# 행렬

In [33]:
import numpy as np
X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

Y = np.dot(X, W) + B

print(X.shape)
print(W.shape)
print(B.shape)
print(Y.shape)

(2,)
(2, 3)
(3,)
(3,)


# broadcast

In [35]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W+B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


# Affine 구현

In [36]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)

# softmax-with-Loss

In [45]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, y):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

In [46]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads # 기울기 구함

In [47]:
# 기울기 확인
# 수치 미분으로 구한 기울기와 오차역전파법으로 구한 기울기가 일치하는지 확인

# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.014565244761837e-10
b1:2.847510872779703e-09
W2:5.151078222327737e-09
b2:1.4074732522934007e-07


In [53]:
# 신경망 구현

# coding: utf-8
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.15071666666666667 0.1602
0.90205 0.9036
0.9237 0.9289
0.9355666666666667 0.9346
0.9442833333333334 0.9406
0.9522166666666667 0.949
0.9575833333333333 0.9535
0.9593833333333334 0.9537
0.96565 0.9598
0.9675 0.9606
0.9702666666666667 0.963
0.97155 0.9627
0.9740833333333333 0.9658
0.9754833333333334 0.965
0.9772833333333333 0.9681
0.9769833333333333 0.966
0.9788666666666667 0.9669
